# Analysis of results using GroupBy function per cluster

In [27]:
# Numerical
import numpy as np
import pandas as pd

# System & Other
import os

# Spatial
import geopandas as gpd

#Plotting
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import display, Markdown, HTML
import matplotlib.pyplot as plt
%matplotlib inline

In [32]:
# Directories
ROOT_DIR = os.path.abspath(os.curdir)
in_path = os.path.join(ROOT_DIR, 'sample_input')
out_path = os.path.join(ROOT_DIR, 'sample_output')

shp_nm = "Ethiopia_vector_clusters_with_attributes.gpkg"

In [33]:
# Create a new geo-dataframe
data_gdf = gpd.read_file(out_path + "\\" + shp_nm)

In [34]:
def calc_LC_sqkm(df, col_list):
    """ 
    This function takes the df where the LC type for different classes is provided per location (row).
    It adds all pixels per location; then is calculates the ratio of LC class in each location (% of total).
    Finally is estimates the area per LC type in each location by multiplying with the total area each row represents.
    
    INPUT: 
    df -> Pandas dataframe with LC type classification 
    col_list -> list of columns to include in the summary (e.g. LC0-LC16)
    
    OUTPUT: Updated dataframe with estimated area (sqkm) of LC types per row
    """
    df["LC_sum"] = df[col_list].sum(axis=1)
    for col in col_list:
        df[col] = df[col]/df["LC_sum"]*df["sqkm"]
    
    return df

In [35]:
lc_cols = ['LC0', 'LC1', 'LC2', 'LC3', 'LC4', 'LC5', 'LC6', 'LC7', 'LC8', 
           'LC9', 'LC10','LC11','LC12', 'LC13', 'LC14', 'LC16' ]

In [36]:
data_gdf_LCsqkm = calc_LC_sqkm(data_gdf, lc_cols)

## Global summaries

In [37]:
sum_cols = ['sqkm', 'WTD','LC0', 'LC1', 'LC2', 'LC3', 'LC4', 'LC5', 'LC6', 'LC7','LC8', 'LC9', 'LC10', 'LC11', 'LC12', 
            'LC13', 'LC14', 'LC15', 'LC16']

sum_rows = ['sum', 'mean', 'min', 'max']

In [38]:
summary_table = pd.DataFrame(index=sum_rows, columns=sum_cols)

In [39]:
# sqkm
summary_table.sqkm[0] = round(data_gdf_LCsqkm.sqkm.sum(),2)
summary_table.sqkm[1] = round(data_gdf_LCsqkm.sqkm.mean(),2)
summary_table.sqkm[2] = round(data_gdf_LCsqkm.sqkm.min(),2)
summary_table.sqkm[3] = round(data_gdf_LCsqkm.sqkm.max(),2)

# Water Table Depth
summary_table.WTD[0] = "NaN"
summary_table.WTD[1] = round(data_gdf_LCsqkm.WTDmean.mean(),2)
summary_table.WTD[2] = round(data_gdf_LCsqkm.WTDmean.min(),2)
summary_table.WTD[3] = round(data_gdf_LCsqkm.WTDmean.max(),2)

# LC0 - Water
summary_table.LC0[0] = round(data_gdf_LCsqkm.LC0.sum(),2)
summary_table.LC0[1] = round(data_gdf_LCsqkm.LC0.mean(),2)
summary_table.LC0[2] = round(data_gdf_LCsqkm.LC0.min(),2)
summary_table.LC0[3] = round(data_gdf_LCsqkm.LC0.max(),2)

# LC1 - Evergreen Needleleaf Forest
summary_table.LC1[0] = round(data_gdf_LCsqkm.LC1.sum(),2)
summary_table.LC1[1] = round(data_gdf_LCsqkm.LC1.mean(),2)
summary_table.LC1[2] = round(data_gdf_LCsqkm.LC1.min(),2)
summary_table.LC1[3] = round(data_gdf_LCsqkm.LC1.max(),2)

# LC2 - Evergreen Broadleaf Forest
summary_table.LC2[0] = round(data_gdf_LCsqkm.LC2.sum(),2)
summary_table.LC2[1] = round(data_gdf_LCsqkm.LC2.mean(),2)
summary_table.LC2[2] = round(data_gdf_LCsqkm.LC2.min(),2)
summary_table.LC2[3] = round(data_gdf_LCsqkm.LC2.max(),2)

# LC3 - Deciduous Needleleaf Forest
summary_table.LC3[0] = round(data_gdf_LCsqkm.LC3.sum(),2)
summary_table.LC3[1] = round(data_gdf_LCsqkm.LC3.mean(),2)
summary_table.LC3[2] = round(data_gdf_LCsqkm.LC3.min(),2)
summary_table.LC3[3] = round(data_gdf_LCsqkm.LC3.max(),2)

# LC4 - Deciduous Broadleaf Forest
summary_table.LC4[0] = round(data_gdf_LCsqkm.LC4.sum(),2)
summary_table.LC4[1] = round(data_gdf_LCsqkm.LC4.mean(),2)
summary_table.LC4[2] = round(data_gdf_LCsqkm.LC4.min(),2)
summary_table.LC4[3] = round(data_gdf_LCsqkm.LC4.max(),2)

# LC5 - Mixed Forests
summary_table.LC5[0] = round(data_gdf_LCsqkm.LC5.sum(),2)
summary_table.LC5[1] = round(data_gdf_LCsqkm.LC5.mean(),2)
summary_table.LC5[2] = round(data_gdf_LCsqkm.LC5.min(),2)
summary_table.LC5[3] = round(data_gdf_LCsqkm.LC5.max(),2)

# LC6 - Mixed Forests
summary_table.LC6[0] = round(data_gdf_LCsqkm.LC6.sum(),2)
summary_table.LC6[1] = round(data_gdf_LCsqkm.LC6.mean(),2)
summary_table.LC6[2] = round(data_gdf_LCsqkm.LC6.min(),2)
summary_table.LC6[3] = round(data_gdf_LCsqkm.LC6.max(),2)

# LC7 - Open Shrublands
summary_table.LC7[0] = round(data_gdf_LCsqkm.LC7.sum(),2)
summary_table.LC7[1] = round(data_gdf_LCsqkm.LC7.mean(),2)
summary_table.LC7[2] = round(data_gdf_LCsqkm.LC7.min(),2)
summary_table.LC7[3] = round(data_gdf_LCsqkm.LC7.max(),2)

# LC8 - Woody Savannas
summary_table.LC8[0] = round(data_gdf_LCsqkm.LC8.sum(),2)
summary_table.LC8[1] = round(data_gdf_LCsqkm.LC8.mean(),2)
summary_table.LC8[2] = round(data_gdf_LCsqkm.LC8.min(),2)
summary_table.LC8[3] = round(data_gdf_LCsqkm.LC8.max(),2)

# LC9 - Savannas
summary_table.LC9[0] = round(data_gdf_LCsqkm.LC9.sum(),2)
summary_table.LC9[1] = round(data_gdf_LCsqkm.LC9.mean(),2)
summary_table.LC9[2] = round(data_gdf_LCsqkm.LC9.min(),2)
summary_table.LC9[3] = round(data_gdf_LCsqkm.LC9.max(),2)

# LC10 - Grasslands
summary_table.LC10[0] = round(data_gdf_LCsqkm.LC10.sum(),2)
summary_table.LC10[1] = round(data_gdf_LCsqkm.LC10.mean(),2)
summary_table.LC10[2] = round(data_gdf_LCsqkm.LC10.min(),2)
summary_table.LC10[3] = round(data_gdf_LCsqkm.LC10.max(),2)

# LC11 - Permanent Wetlands
summary_table.LC11[0] = round(data_gdf_LCsqkm.LC11.sum(),2)
summary_table.LC11[1] = round(data_gdf_LCsqkm.LC11.mean(),2)
summary_table.LC11[2] = round(data_gdf_LCsqkm.LC11.min(),2)
summary_table.LC11[3] = round(data_gdf_LCsqkm.LC11.max(),2)

# LC12 - Croplands
summary_table.LC12[0] = round(data_gdf_LCsqkm.LC12.sum(),2)
summary_table.LC12[1] = round(data_gdf_LCsqkm.LC12.mean(),2)
summary_table.LC12[2] = round(data_gdf_LCsqkm.LC12.min(),2)
summary_table.LC12[3] = round(data_gdf_LCsqkm.LC12.max(),2)

# LC13 - Urban and Built-Up
summary_table.LC13[0] = round(data_gdf_LCsqkm.LC13.sum(),2)
summary_table.LC13[1] = round(data_gdf_LCsqkm.LC13.mean(),2)
summary_table.LC13[2] = round(data_gdf_LCsqkm.LC13.min(),2)
summary_table.LC13[3] = round(data_gdf_LCsqkm.LC13.max(),2)

# LC14 - Cropland/Natural Vegetation Mosaic
summary_table.LC14[0] = round(data_gdf_LCsqkm.LC14.sum(),2)
summary_table.LC14[1] = round(data_gdf_LCsqkm.LC14.mean(),2)
summary_table.LC14[2] = round(data_gdf_LCsqkm.LC14.min(),2)
summary_table.LC14[3] = round(data_gdf_LCsqkm.LC14.max(),2)

# LC15 - Snow and Ice
summary_table.LC15[0] = "NaN"
summary_table.LC15[1] = "NaN"
summary_table.LC15[2] = "NaN"
summary_table.LC15[3] = "NaN"

# LC16 - Snow and Ice
summary_table.LC16[0] = round(data_gdf_LCsqkm.LC16.sum(),2)
summary_table.LC16[1] = round(data_gdf_LCsqkm.LC16.mean(),2)
summary_table.LC16[2] = round(data_gdf_LCsqkm.LC16.min(),2)
summary_table.LC16[3] = round(data_gdf_LCsqkm.LC16.max(),2)

In [40]:
display(Markdown('### Country Summary \n These are the summarized results for Ethiopia'))
display(summary_table)
display(Markdown('### Class Description \n ##### Percentage \n LC0 - Water \n\n LC1 - Evergreen Needleleaf Forest \n\n LC2 - Evergreen Broadleaf Forest \n\n  LC3 - Deciduous Needleleaf Forest \n\n LC4 - Deciduous Broadleaf Forest \n\n LC5 - Mixed Forests \n\n LC6 - Closed Shrublands \n\n LC7 - Open Shrublands \n\n LC8 - Woody Savannas \n\n LC9 - Savannas \n\n LC10 - Grasslands \n\n LC11 - Permanent Wetlands \n\n LC12 - Croplands \n\n LC13 - Urban and Built-Up \n\n LC14 - Cropland/Natural Vegetation Mosaic \n\n LC15 - Snow and Ice \n\n LC16 - Barren or Sparsely Vegetated'))

### Country Summary 
 These are the summarized results for Ethiopia

,sqkm,WTD,LC0,LC1,LC2,LC3,LC4,LC5,LC6,LC7,LC8,LC9,LC10,LC11,LC12,LC13,LC14,LC15,LC16
sum,1.13233e+06,NaN,6245.23,0.21,28762.2,7.56,1337.15,1072.09,31798.6,312018,104542,165110,168541,926.69,127635,736.14,117277,NaN,66318.8
mean,82.35,42.61,30.76,0.21,25.23,0.33,1.81,1.04,16.82,45.12,20.54,25.05,22.61,1.63,18.9,2.46,16.07,NaN,30.14
min,0,0,0.17,0.21,0.19,0.19,0.18,0.15,0.01,0,0.18,0,0.01,0.19,0.08,0.13,0.03,NaN,0
max,84.53,149.42,84.53,0.21,84.53,1.73,31.7,15.85,84.11,84.53,84.53,84.53,84.53,39.05,84.53,47.55,84.53,NaN,84.53


### Class Description 
 ##### Percentage 
 LC0 - Water 

 LC1 - Evergreen Needleleaf Forest 

 LC2 - Evergreen Broadleaf Forest 

  LC3 - Deciduous Needleleaf Forest 

 LC4 - Deciduous Broadleaf Forest 

 LC5 - Mixed Forests 

 LC6 - Closed Shrublands 

 LC7 - Open Shrublands 

 LC8 - Woody Savannas 

 LC9 - Savannas 

 LC10 - Grasslands 

 LC11 - Permanent Wetlands 

 LC12 - Croplands 

 LC13 - Urban and Built-Up 

 LC14 - Cropland/Natural Vegetation Mosaic 

 LC15 - Snow and Ice 

 LC16 - Barren or Sparsely Vegetated

## Cluster Summaries

In [41]:
data_gdf_LCsqkm["cluster"] = data_gdf_LCsqkm["cluster"].astype(str)
non_clustered_data = data_gdf_LCsqkm[data_gdf_LCsqkm["cluster"] == "None"]

display(Markdown('**Note** that there are {} polygons that are not assigned to a cluster  -- classified as "None"'
                 .format(len(non_clustered_data))))

**Note** that there are 11 polygons that are not assigned to a cluster  -- classified as "None"

#### Groupby on clusters

In [42]:
clusters = data_gdf_LCsqkm.groupby(['cluster'])

In [43]:
clust_sum_cols = ['sqkm','LC0', 'LC1', 'LC2', 'LC3', 'LC4', 'LC5', 'LC6', 'LC7','LC8', 'LC9', 'LC10', 'LC11', 'LC12', 
            'LC13', 'LC14', 'LC16']

In [44]:
clusters_sum = clusters[clust_sum_cols].sum().sort_values(ascending=False, by='sqkm').reset_index()
display(Markdown('#### Cluster summary statistics for area and land cover'))
clusters_sum

#### Cluster summary statistics for area and land cover

,cluster,sqkm,LC0,LC1,LC2,LC3,LC4,LC5,LC6,LC7,LC8,LC9,LC10,LC11,LC12,LC13,LC14,LC16
0,OROC03,176554.991737,1066.678518,0.000000,1119.766368,0.845349,463.766704,165.720116,7921.802325,20306.994176,20633.607077,16510.932581,89234.981742,112.637706,5519.931463,150.468154,12520.151510,826.707949
1,SOMC01,147287.381301,0.000000,0.000000,0.000000,0.000000,194.218973,0.422675,2995.525189,130944.768367,32.334606,12.257563,778.700842,0.845349,34.645058,12.426511,362.282552,11918.953617
2,SOMC03,105956.069594,0.000000,0.000000,0.000000,0.000000,18.960928,0.201274,2051.472427,97549.689911,18.103276,21.390862,2084.444012,0.000000,153.087997,31.540921,387.389244,3639.788744
3,AMHC02,104663.627409,3004.832071,0.000000,13.013890,4.226746,1.572819,63.584518,6.687784,506.573923,1129.834970,52765.713629,1548.458477,88.573510,34191.128772,44.239079,11218.683123,76.504100
4,OROC01,83520.498584,115.140428,0.207193,8823.129582,1.434981,3.993572,457.979139,44.075510,59.421472,12110.563255,2962.961055,15031.621920,173.512459,19387.805676,294.037490,24042.007921,12.606930
5,OROC02,81153.520891,992.439933,0.000000,6844.224684,0.634012,3.712589,62.548991,25.553611,438.274359,34053.079566,12248.516244,6344.103105,84.871759,4768.410977,43.600573,15149.716730,93.833759
6,AFAC02,73307.408156,545.824211,0.000000,1.179950,0.000000,0.791223,8.985960,22.914632,14448.320228,3.951405,110.897351,11158.812167,46.326281,2233.114393,12.211897,1184.101766,43529.976691
7,SOMC02,57426.231294,0.000000,0.000000,0.000000,0.000000,627.680120,8.977254,18604.674762,20400.635728,581.177837,160.169515,12417.224937,0.422675,119.445073,9.198701,565.664430,3930.960261
8,SOUC03,52576.812252,446.176903,0.000000,3138.183659,0.000000,5.917444,8.030817,19.435486,2823.849529,14504.987880,7796.137665,11286.542720,94.647401,1144.159106,4.015409,11271.145378,33.582854
9,AMHC01,50720.950557,0.000000,0.000000,4.072103,0.000000,0.390464,128.609691,53.627591,290.490318,357.878105,10259.224588,2655.791131,22.987285,31373.064328,33.807830,5540.038531,0.968592


In [45]:
clusters_mean = clusters['WTDmean'].mean().sort_values(ascending=False).reset_index()
display(Markdown('#### Cluster summary statistics for Water Table Depth (in m)'))
clusters_mean

#### Cluster summary statistics for Water Table Depth (in m)

,cluster,WTDmean
0,TIGC01,77.574067
1,TIGC02,60.996776
2,AMHC01,58.320362
3,AFAC01,55.809983
4,AMHC02,49.700275
5,BENC02,49.591887
6,AFAC02,49.144369
7,SOMC02,46.051644
8,OROC03,45.208581
9,SOUC02,43.197206


### Explore particular clusters

In [46]:
clusters.get_group('BENC01')[clust_sum_cols].sum()

sqkm    20476.506930
LC0         0.000000
LC1         0.000000
LC2         0.000000
LC3         0.000000
LC4         4.934247
LC5        10.871269
LC6         0.000000
LC7         0.000000
LC8      1286.932838
LC9     19127.767279
LC10        1.389071
LC11        0.000000
LC12       32.228828
LC13        0.000000
LC14       12.383398
LC16        0.000000
dtype: float64

In [47]:
clusters.get_group('BENC01')[clust_sum_cols].describe()

,sqkm,LC0,LC1,LC2,LC3,LC4,LC5,LC6,LC7,LC8,LC9,LC10,LC11,LC12,LC13,LC14,LC16
count,271.000000,0.0,0.0,0.0,0.0,18.000000,27.000000,0.0,0.0,112.000000,271.000000,2.000000,0.0,56.000000,0.0,24.000000,0.0
mean,75.559066,NaN,NaN,NaN,NaN,0.274125,0.402640,NaN,NaN,11.490472,70.582167,0.694536,NaN,0.575515,NaN,0.515975,NaN
std,21.986161,NaN,NaN,NaN,NaN,0.180215,0.326629,NaN,NaN,15.585140,23.325456,0.140317,NaN,0.790994,NaN,0.585562,NaN
min,0.097967,NaN,NaN,NaN,NaN,0.193887,0.194532,NaN,NaN,0.193887,0.097967,0.595316,NaN,0.183699,NaN,0.193687,NaN
25%,84.534918,NaN,NaN,NaN,NaN,0.196140,0.197052,NaN,NaN,0.790046,67.535153,0.644926,NaN,0.198439,NaN,0.198439,NaN
50%,84.534918,NaN,NaN,NaN,NaN,0.198439,0.200320,NaN,NaN,4.569731,83.368919,0.694536,NaN,0.392276,NaN,0.199847,NaN
75%,84.534918,NaN,NaN,NaN,NaN,0.199375,0.398750,NaN,NaN,15.031737,84.534918,0.744145,NaN,0.443631,NaN,0.448241,NaN
max,84.534918,NaN,NaN,NaN,NaN,0.782731,1.572743,NaN,NaN,63.799938,84.534918,0.793755,NaN,4.186871,NaN,2.443174,NaN


In [48]:
#variable = "LC12"     #  Cropland
#
#fig, ax = plt.subplots(figsize=(10, 10))
#clusters[variable].sum().sort_values(ascending=False).reset_index().plot.bar(ax=ax, 
#                                                                                                       x='cluster', 
#                                                                                                       y=variable)
#txt = ax.set_title('{} per cluster in Ethiopia'.format(variable))

### Interactive graphs

In [49]:
# Get cluster names
clust_names = list(data_gdf_LCsqkm.cluster.unique())

# Explude non clustered data
clust_names.remove("None")

In [50]:
# Create a dictionary that includes the name of the clusters and a selected parameter
clust_dict = dict.fromkeys(clust_names, 1)

# Update the values based on parameter
parameter = "sqkm"                        # This is the column you want to include in the graph

for key, value in clust_dict.items():
    clust_dict[key] = round(clusters.get_group(key)[parameter].sum(),2)   # Note that this calculates the sum, change if needed

fig_Cluster = px.bar(pd.DataFrame.from_dict(clust_dict, orient='index', columns=['Sum']), title="Dictribution of {} over clusters in Ethiopia".format(parameter))
fig_Cluster.show()

In [51]:
fig_Cluster.write_html("Total area (sqkm) per cluster.html")

### Export data - tables - summary - graphs as needed

In [22]:
#data_gdf_clusters.to_csv("testFun.csv")